In [4]:
# For data
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
from datetime import datetime, timedelta
import pytz
from pytz import timezone
from pandas.io import sql
import pymysql.cursors

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from __future__ import division

###### Please find below questions related to "demographics" part of the study

- Users characteristics ?
- What does a typical user look like ?
- What goals do they have ?
- How do these goals vary by demographics or need ? 

In [5]:
#Importing all datasets (all of operations are detailled in notebooks from directory called "exploring")

#users with complete timezone
users_df = pd.read_csv('/Users/kevenlemoing/Sites/Lifesum_assignement/data/users_timezone.csv',error_bad_lines=False)

#food
food_df = pd.read_csv('/Users/kevenlemoing/Sites/Lifesum_assignement/data/food.csv',error_bad_lines=False)

#exercise
ex_df = pd.read_csv('/Users/kevenlemoing/Sites/lifesum_assignement/data/exercise.csv',error_bad_lines=False)

#weight
weight_df = pd.read_csv('/Users/kevenlemoing/Sites/lifesum_assignement/data/weight.csv',error_bad_lines=False)